# Detect fires at a location
A simplified version of Krishna's `s2_fires_exploration` notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

from scripts import dl_utils

In [ ]:
def fire_detection(img):
    '''
    run fire detection on an L8/S2 image. 
    assumes stack has been rasterized with raw unscaled values
    assumes band ordering:
        bands = [
            'coastal-aerosol',
            'blue',
            'green',
            'red',
            'nir',
            'swir1',
            'swir2',
            'cloud-mask'
        ]
    
    Algorithm adapted from paper:
        Active fire detection using Landsat-8/OLI data
        Schroeder 2015
    '''
    img = img.astype(int)
    cond1 = (img[:, :,11] / img[:, :, 7]) > 2.5
    cond2 = (img[:, :,11] - img[:, :, 7]) > 0.3
    cond3 = (img[:, :,11] > 0.5)
    
    check1 = (cond1 & cond2 & cond3)
    
    cond4 = (img[:, :, 10] > 0.8)
    cond5 = (img[:, :, 0] < 0.2)
    cond6 = (img[: ,:, 7] > 0.4)
    cond7 = (img[:, :, 11] < 0.1)
    
    check2 = (cond4 & cond5 & (cond6 | cond7))
    
    fire_pix = np.logical_or(check1, check2)
    fire_pix[img.mask[:,:,0]] = 0
    
    # NOT FROM PAPER: throw on a light dilation of fire pixels
    fire_pix = cv2.dilate(fire_pix.astype(np.uint8), np.ones((3, 3), np.uint8), iterations=1)
        
    return fire_pix

In [ ]:
def new_fire_algo(p):
    p = p.astype(int)
    # Check red/SWIR2 ratio
    condition_1 = p[:,:,3] <= 1.045 * p[:,:,11] - 0.071 * 3000
    # Check SWIR2/SWIR1 ratio
    condition_2 = p[:,:,11] / p[:,:, 10] > 1
    fire_pixels = np.logical_and(condition_1, condition_2)
    fire_pixels[p.mask[:,:,0]] = 0
    return fire_pixels

In [ ]:
from scripts import viz_tools
viz_tools.band_descriptions

In [ ]:
threshold = 0.1
for p in patches:
    p = p.astype(int)
    sahm = (p[:,:,11] - p[:,:,10]) / (p[:,:,11] + p[:,:,10])
    sahm = p[:,:,11] > 3000
    sahm[p.mask[:,:,0]] = 0
    if np.sum(sahm > threshold) > 0:
        plt.figure(figsize=(8,5), dpi=150)
        plt.subplot(1,2,1)
        plt.title(f'RGB')
        rgb = np.clip(p[:,:,3:0:-1] / 3000, 0, 1)
        plt.imshow(rgb)
        plt.axis('off')
        plt.subplot(1,2,2)
        fire_pix = sahm > threshold
        fire_rgb = rgb.copy()
        fire_rgb[fire_pix > 0,0] = 1
        fire_rgb[fire_pix > 0,1] = .2
        fire_rgb[fire_pix > 0,2] = .2
        plt.title(f'Fire - {np.sum(fire_pix)}')
        plt.imshow(fire_rgb)
        plt.axis('off')
        plt.show()

In [ ]:
p = p.astype(int)

In [ ]:
def fire_sentinel_hub(p):
    NGDR = (p[:,:,1] - p[:,:,2]) / (p[:,:,1] + p[:,:,2])
    inverse = (p[:,:,1] - 0.2 * np.ma.max(p[:,:,1])) / (0.5 - 0.2)
    SAHM_index = (p[:,:,11] - p[:,:,10]) / (p[:,:,11] + p[:,:,10])

    if inverse > 1:
        return 0.5 * p[:,:,]

var NGDR = index(B02, B03);
var Inverse = (B02 - 0.2) / (0.5 - 0.2);
//Fire indicator
var SAHM_INDEX= ((B12 - B11) / (B12 + B11));

if (Inverse > 1) { 
    return [0.5 * B04, 0.5 * B03, 20 * B02 ];
}

if (Inverse > 0 && NGDR>0) { 
    return [0.5 * B04  , 0.5 * B03, 20 * B02];
}

if((SAHM_INDEX>0.4)||(B12>1)){
  return[20*B04, 1*B03, 1*B02];
}

else {
 return [B04,B04,B04]
}

//Red color indicates active fire areas and points
//The blue range is a cloud mask

In [ ]:
rect_width = 0.01
#coords = [115.2206873996286, -8.72220352727478]
coords = [115.16999128605232, -8.090578007584227]
start_date = '2016-06-01'
end_date = '2021-12-01'
patches, raster_info = dl_utils.download_patch(dl_utils.rect_from_point(coords, rect_width), start_date, end_date)

In [ ]:
threshold = 0.3
for p in patches:
    p = p.astype(int)
    sahm = (p[:,:,11] - p[:,:,10]) / (p[:,:,11] + p[:,:,10])
    #sahm = p[:,:,11] > 3000
    sahm[p.mask[:,:,0]] = 0
    if np.sum(sahm > threshold) > 0:
        plt.figure(figsize=(8,5), dpi=150)
        plt.subplot(1,2,1)
        plt.title(f'RGB')
        rgb = np.clip(p[:,:,3:0:-1] / 3000, 0, 1)
        plt.imshow(rgb)
        plt.axis('off')
        plt.subplot(1,2,2)
        fire_pix = sahm > threshold
        fire_rgb = rgb.copy()
        fire_rgb[fire_pix > 0,0] = 1
        fire_rgb[fire_pix > 0,1] = .2
        fire_rgb[fire_pix > 0,2] = .2
        plt.title(f'Fire - {np.sum(fire_pix)}')
        plt.imshow(fire_rgb)
        plt.axis('off')
        plt.show()

In [ ]:
fire_index = []
for index, p in enumerate(patches):
    fire_pix_new = new_fire_algo(p)
    fire_pix_old = fire_detection(p)
    fire_pixels = np.logical_and(fire_pix_new > 0, fire_pix_old > 0)
    if np.sum(fire_pixels):
        print(index, np.sum(fire_pixels))
        fire_index.append(index)

In [ ]:
#fire_pixels = np.array([np.sum(fire_detection(p)) for p in patches])
fire_index = []
for index, p in enumerate(patches):
    fire_pix_new = new_fire_algo(p)
    fire_pix_old = fire_detection(p)
    if np.sum(np.logical_and(fire_pix_new > 0, fire_pix_old > 0)):
        fire_index.append(index)
for index in fire_index:
    plt.figure(figsize=(8,5), dpi=150)
    plt.subplot(1,2,1)
    plt.title(f'RGB - {index}')
    rgb = np.clip(patches[index][:,:,3:0:-1] / 3000, 0, 1)
    plt.imshow(rgb)
    plt.axis('off')
    plt.subplot(1,2,2)
    fire_pix_new = new_fire_algo(patches[index])
    fire_pix_old = fire_detection(patches[index])
    fire_pix = np.logical_and(fire_pix_new > 0, fire_pix_old > 0)
    fire_rgb = rgb.copy()
    fire_rgb[fire_pix > 0,0] = 1
    fire_rgb[fire_pix > 0,1] = .2
    fire_rgb[fire_pix > 0,2] = .2
    plt.title(f'Fire - {np.sum(fire_pix)}')
    plt.imshow(fire_rgb)
    plt.axis('off')
    plt.show()

In [ ]:
plt.imshow(patches[173][:,:,11])

In [ ]:
np.sum(fire_detection(p))

In [ ]:
np.sum(fire_detection(p))

In [ ]:
for p in patches:
    plt.subplot(1,2,1)
    plt.title('RGB')
    plt.imshow(np.clip(p[:,:,3:0:-1] / 3000, 0, 1))
    plt.axis('off')
    plt.subplot(1,2,2)
    fire_pix = fire_detection(p)
    plt.title(f'Fire - {np.sum(fire_pix)}')
    plt.imshow(fire_pix)
    plt.axis('off')
    plt.show()